# Build the index of the websites using FAISS

In [1]:
!pip install flask sentence-transformers faiss-cpu beautifulsoup4 requests pydantic pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [4]:
import os
import csv
import time
import random
import logging
import numpy as np
import faiss
import json
import requests
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import pickle

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s:%(name)s: %(message)s')
logger = logging.getLogger(__name__)

# Embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to get embedding
def get_embedding(text):
    return model.encode(text)

# Function to split text into chunks
def split_text(text, chunk_size=1000, chunk_overlap=200):
    words = text.split()
    chunks = []

    for i in range(0, len(words), chunk_size - chunk_overlap):
        chunk = ' '.join(words[i:i + chunk_size])
        chunks.append(chunk)

    return chunks

# Define a list of user agents for rotation
headers_list = [
    {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Referer': 'https://www.google.com/',
        'DNT': '1',
    },
    {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Safari/605.1.15',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Referer': 'https://www.bing.com/',
        'DNT': '1',
    },
    {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:78.0) Gecko/20100101 Firefox/78.0',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Referer': 'https://duckduckgo.com/',
        'DNT': '1',
    },
    {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
        'Cache-Control': 'max-age=0',
    }
]

# Improved website scraping function with rate limiting
def scrape_website(url, headers_list):
    try:
        # Rotate headers
        headers = random.choice(headers_list)

        # Add a random delay between requests (2-5 seconds)
        time.sleep(random.uniform(2, 5))

        response = requests.get(url, headers=headers, timeout=15)

        # Check if we're getting blocked
        if response.status_code == 429:
            logger.error(f"Rate limited at {url}. Waiting longer before retry...")
            time.sleep(random.uniform(10, 15))  # Wait longer before potential retry
            # Try one more time with different headers
            headers = random.choice(headers_list)
            response = requests.get(url, headers=headers, timeout=15)
            if response.status_code == 429:
                logger.error(f"Still rate limited at {url} after retry. Skipping.")
                return None

        response.raise_for_status()

        # Check if we got a reasonable amount of content
        if len(response.text) < 1000:  # Arbitrary threshold
            logger.warning(f"Response from {url} seems too small, might be blocked")
            return None

        soup = BeautifulSoup(response.text, 'html.parser')

        # Remove script and style elements
        for script in soup(["script", "style"]):
            script.extract()

        text = soup.get_text()

        # Clean up text
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = '\n'.join(chunk for chunk in chunks if chunk)

        return text

    except requests.exceptions.RequestException as e:
        logger.error(f"Failed to scrape {url}: {e}")
        return None

# Function to create and save FAISS index
def create_faiss_index(urls, output_dir="faiss_index", max_websites=100, chunk_size=1000, chunk_overlap=200):
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Initialize FAISS index
    embedding_dim = model.get_sentence_embedding_dimension()
    faiss_index = faiss.IndexFlatL2(embedding_dim)

    # For storing metadata
    metadata = []

    # Process websites
    processed_websites = 0
    skipped_websites = 0

    for url in tqdm(urls, desc="Processing websites"):
        if processed_websites >= max_websites:
            break

        logger.info(f"Processing website {processed_websites + 1}/{max_websites}: {url}")

        # Skip certain problematic sites that are known to block scraping
        if "google.com/maps" in url or "maps.google.com" in url:
            logger.warning(f"Skipping Google Maps URL as it requires JavaScript: {url}")
            skipped_websites += 1
            continue

        # Use improved scraping function
        text = scrape_website(url, headers_list)

        if text is None or len(text.strip()) < 100:
            logger.warning(f"Skipping {url} due to lack of content or scraping error.")
            skipped_websites += 1
            continue

        # Split into chunks
        chunks = split_text(text, chunk_size, chunk_overlap)

        chunks_added = 0
        for i, chunk in enumerate(chunks):
            if len(chunk) < 50:  # Skip very small chunks
                continue

            # Generate embedding
            embedding = get_embedding(chunk)

            # Add to FAISS index
            faiss_index.add(np.array([embedding]).astype('float32'))

            # Store metadata
            metadata.append({
                "text": chunk,
                "url": url,
                "chunk_id": i
            })

            chunks_added += 1

        logger.info(f"Added {chunks_added} chunks from {url}")
        processed_websites += 1

        # Add a longer delay between different websites to avoid mass rate limiting
        time.sleep(random.uniform(3, 7))

    logger.info(f"Processed {processed_websites} websites, skipped {skipped_websites} websites")

    # Save FAISS index
    faiss.write_index(faiss_index, os.path.join(output_dir, "faiss.index"))

    # Save metadata
    with open(os.path.join(output_dir, "metadata.json"), 'w') as f:
        json.dump(metadata, f)

    # Also save as pickle for backup
    with open(os.path.join(output_dir, "metadata.pkl"), 'wb') as f:
        pickle.dump(metadata, f)

    logger.info(f"Index saved to {output_dir}")
    logger.info(f"Total chunks indexed: {len(metadata)}")

    return faiss_index, metadata

# Function to search the index
def search_index(query, faiss_index, metadata, k=5):
    # Generate query embedding
    query_embedding = get_embedding(query)

    # Search
    distances, indices = faiss_index.search(np.array([query_embedding]).astype('float32'), k)

    results = []
    for i, idx in enumerate(indices[0]):
        if idx < len(metadata):  # Ensure index is valid
            results.append({
                "text": metadata[idx]["text"],
                "url": metadata[idx]["url"],
                "distance": distances[0][i]
            })

    return results



In [8]:
# Example usage
if __name__ == "__main__":
    # List of URLs to process
    urls = [
        "https://en.wikipedia.org/wiki/Artificial_intelligence",
        "https://en.wikipedia.org/wiki/Machine_learning",
        "https://en.wikipedia.org/wiki/Natural_language_processing",
        "https://en.wikipedia.org/wiki/Computer_vision",
        "https://en.wikipedia.org/wiki/Neural_network",
        # Add more URLs as needed
        "https://www.google.com/",
        "https://www.bing.com/",
        "https://duckduckgo.com/",
        "https://www.youtube.com/",
        "https://www.amazon.com/",
        "https://www.yahoo.com",
        "https://finance.yahoo.com",
        "https://www.stockanalysis.com",
    ]

    # If you have a file with URLs
    # with open('urls.txt', 'r') as f:
    #     urls = [line.strip() for line in f if line.strip()]

    #max_websites = len(urls)

    # Create index
    faiss_index, metadata = create_faiss_index(urls, max_websites=13)

    # Example search
    results = search_index("What is machine learning?", faiss_index, metadata)

    print(f"Found {len(results)} results:")
    # Print results
    for i, result in enumerate(results):
        print(f"Result {i+1}:")
        print(f"URL: {result['url']}")
        print(f"Distance: {result['distance']}")
        print(f"Text: {result['text'][:200]}...")
        print("-" * 50)

Processing websites: 100%|██████████| 13/13 [02:08<00:00,  9.89s/it]

Found 5 results:
Result 1:
URL: https://en.wikipedia.org/wiki/Machine_learning
Distance: 0.5439536571502686
Text: machine learning approaches in performance.[2] ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medici...
--------------------------------------------------
Result 2:
URL: https://en.wikipedia.org/wiki/Artificial_intelligence
Distance: 0.6441100835800171
Text: model that describes the probability that a particular action will change the state in a particular way and a reward function that supplies the utility of each state and the cost of each action. A pol...
--------------------------------------------------
Result 3:
URL: https://en.wikipedia.org/wiki/Machine_learning
Distance: 0.7451267838478088
Text: machine learning (RBML) is a branch of machine learning that automatically discovers and learns 'rules' from data. It provides interpretable models, making it useful for decis